# Introduction

* This is the demo notebook on how to use the [Cassava Trained Models](http://www.kaggle.com/dataset/80c06af6e1386ee747dc16103dd54a802680014cb20502f94263e673aeeb911d)

* Upvote the [Dataset](http://www.kaggle.com/dataset/80c06af6e1386ee747dc16103dd54a802680014cb20502f94263e673aeeb911d) and to go forward from here, click the blue "Edit Notebook" button at the top of the kernel. This will create a copy of the code and environment for you to edit. Delete, modify, and add code as you please. Happy Kaggling!

* Before running the kernel turn on the **GPU(cuda)**.

# Import Packages

In [ ]:
import os
import sys
sys.path.append('../input/vision-transformer-pytorch/VisionTransformer-Pytorch')
sys.path.append('../input/pytorch-image-models/pytorch-image-models-master')
sys.path.append('../input/efficientnet-pytorch')

import glob
import numpy as np
import pandas as pd
from tqdm import tqdm
from collections import defaultdict

from PIL import Image
import albumentations as A
from albumentations.pytorch import ToTensorV2

import timm
from vision_transformer_pytorch import VisionTransformer
from efficientnet_pytorch import EfficientNet

import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Directory Settings

In [ ]:
# Config settings

class CFG:
    img_size = 512
    num_classes = 5
    num_workers = 4
    batch_size = 64
    epochs = 1


# Directory settings

OUTPUT_DIR = './'

ROOT_DIR = '../input/cassava-leaf-disease-classification/'

TRAIN_PATH = '../input/cassava-leaf-disease-classification/train_images'

TEST_PATH = '../input/cassava-leaf-disease-classification/test_images'

MODEL_DIR = '../input/cassava-models'

# Define Dataset

* In this cell, there are two different augmentations are defined. One for direct inference and other is for **TTA(Test Time Augmentation)**.

In [ ]:
def get_augmentation(data):

    if data=='test':
        return A.Compose([
            A.Resize(CFG.img_size, CFG.img_size),
            A.Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225],
                max_pixel_value=255.0,
                p=1.0),
            ToTensorV2()
        ])


# augmentations taken from: https://www.kaggle.com/khyeh0719/pytorch-efficientnet-baseline-inference-tta
test_aug = A.Compose([
    A.Resize(CFG.img_size, CFG.img_size),
    A.Transpose(p=0.5),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.HueSaturationValue(
        hue_shift_limit=0.2, 
        sat_shift_limit=0.2,
        val_shift_limit=0.2, 
        p=0.5
    ),
    A.RandomBrightnessContrast(
        brightness_limit=(-0.1,0.1), 
        contrast_limit=(-0.1, 0.1), 
        p=0.5
    ),
    A.Normalize(
        mean=[0.485, 0.456, 0.406], 
        std=[0.229, 0.224, 0.225], 
        max_pixel_value=255.0, 
        p=1.0
    ),
    ToTensorV2()
], p=1.)

In [ ]:
class TestDataset(Dataset):

    def __init__(self, df, transform=None):
        self.df = df
        self.file_names = df['image_id'].values
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        file_name = self.file_names[idx]
        file_path = f'{TEST_PATH}/{file_name}'
        image = Image.open(file_path).convert('RGB')
        image = np.array(image)
        if self.transform:
            augmented = self.transform(image=image)
            image = augmented['image']
        return image

# Model

* In this section, different model calss are defined which can be replace or add with other model class.
* After that, some functions are created to load and initialize the models.
* And at the end of this section, **Ensemble class** is created to ensemble the models and perform mean prediction(make prediction for every model and took mean of the predictions).

In [ ]:
# ----- CustomResNext ----- #

class CustomResNext(nn.Module):

    def __init__(self, pretrained=False):
        super().__init__()
        self.model = timm.create_model('resnext50_32x4d', pretrained=pretrained)
        n_features = self.model.fc.in_features
        self.model.fc = nn.Linear(n_features, CFG.num_classes)

    def forward(self, x):
        x = self.model(x)
        return x


# ----- Efficientnet-b4 with_torch_CrossEntorpy ----- #

class LeafModel(nn.Module):

    def __init__(self):
        super().__init__()
        self.effnet = EfficientNet.from_name("efficientnet-b4")
        self.dropout = nn.Dropout(0.1)
        self.out = nn.Linear(1792, 5)

    def forward(self, image):
        batch_size,_,_,_ = image.shape
        x = self.effnet.extract_features(image)
        x = F.adaptive_avg_pool2d(x, 1).reshape(batch_size, -1)
        x = self.out(self.dropout(x))
        return x


class Efficient_b4(nn.Module):

    def __init__(self, PATH, pretrained=False):
        super().__init__()
        self.model = timm.create_model('tf_efficientnet_b4_ns', pretrained=pretrained)
        n_features = self.model.classifier.in_features
        self.model.classifier = nn.Linear(n_features, CFG.num_classes)
        checkpoint = torch.load(PATH)
        self.model.load_state_dict(checkpoint['model_state_dict'])

    def forward(self, x):
        x = self.model(x)
        return x


class Efficient_b7(nn.Module):

    def __init__(self, PATH, pretrained=False):
        super().__init__()
        self.model = timm.create_model('tf_efficientnet_b7_ns', pretrained=pretrained)
        n_features = self.model.classifier.in_features
        self.model.classifier = nn.Linear(n_features, CFG.num_classes)
        checkpoint = torch.load(PATH)
        self.model.load_state_dict(checkpoint['model_state_dict'])

    def forward(self, x):
        x = self.model(x)
        return x


# ----- Vision Transformer ----- #

class VitModel(nn.Module):

    def __init__(self,PATH):
        super().__init__()
        self.model = VisionTransformer.from_name('ViT-B_16', num_classes=5)
        self.model.load_state_dict(torch.load(PATH))

    def forward(self, image):
        batch_size,_,_,_ = image.shape
        image = F.interpolate(image,(384,384))
        x = self.model(image)
        return x

In [ ]:
# Efficientnet-b4 with_torch_CrossEntorpy
def get_leaf_model(PATH):
    model = LeafModel()
    checkpoint = torch.load(PATH)
    model.load_state_dict(checkpoint['model_state_dict'])
    model.eval()
    return model.to(device)


def get_resnext_model(PATH):
    model = CustomResNext()
    model.load_state_dict(torch.load(PATH))
    model.eval()
    return model.to(device)


def get_efficient_b4_model(PATH):
    model = Efficient_b4(PATH)
    model.eval()
    return model.to(device)


def get_efficient_b7_model(PATH):
    model = Efficient_b7(PATH)
    model.eval()
    return model.to(device)

def get_vit_model(PATH):
    model = VitModel(PATH)
    model.eval()
    return model.to(device)

In [ ]:
# EnsembledModel function

class EnsembledModel():

    def __init__(self, model_paths):
        super().__init__()
        self.num_models = len(model_paths)

        self.leafmodel1 = get_leaf_model(model_paths[0])
        self.leafmodel2 = get_leaf_model(model_paths[1])

        self.effb4_model1 = get_efficient_b4_model(model_paths[2])
        self.effb4_model2 = get_efficient_b4_model(model_paths[3])
        self.effb4_model3 = get_efficient_b4_model(model_paths[4])
        self.effb4_model4 = get_efficient_b4_model(model_paths[5])
        self.effb4_model5 = get_efficient_b4_model(model_paths[6])
        
        self.effb7_model1 = get_efficient_b7_model(model_paths[7])
        self.effb7_model2 = get_efficient_b7_model(model_paths[8])
        self.effb7_model3 = get_efficient_b7_model(model_paths[9])

        self.res50_model1 = get_resnext_model(model_paths[10])
        self.res50_model2 = get_resnext_model(model_paths[11])
        self.res50_model3 = get_resnext_model(model_paths[12])
        self.res50_model4 = get_resnext_model(model_paths[13])
        self.res50_model5 = get_resnext_model(model_paths[14])

        self.res50_lr_model1 = get_resnext_model(model_paths[15])
        self.res50_lr_model2 = get_resnext_model(model_paths[16])
        self.res50_lr_model3 = get_resnext_model(model_paths[17])


    def predict(self, x):
        with torch.no_grad():
            l1 = self.leafmodel1(x)
            l2 = self.leafmodel2(x)

            b4_e1 = self.effb4_model1(x)
            b4_e2 = self.effb4_model2(x)
            b4_e3 = self.effb4_model3(x)
            b4_e4 = self.effb4_model4(x)
            b4_e5 = self.effb4_model5(x)

            r1 = self.res50_model1(x)
            r2 = self.res50_model2(x)
            r3 = self.res50_model3(x)
            r4 = self.res50_model4(x)
            r5 = self.res50_model5(x)

            b7_e1 = self.effb7_model1(x)
            b7_e2 = self.effb7_model2(x)
            b7_e3 = self.effb7_model3(x)

            lr_r1 = self.res50_lr_model1(x)
            lr_r2 = self.res50_lr_model2(x)
            lr_r3 = self.res50_lr_model3(x)

            pred = (l1+l2 + b4_e1+b4_e2+b4_e3+b4_e4+b4_e5 + b7_e1+b7_e2+b7_e3 + r1+r2+r3+r4+r5 + lr_r1+lr_r2+lr_r3) / (self.num_models)

            return pred

In [ ]:
model_paths = [
    '../input/cassava-trained-models/with_torch_crossentropy_LeafDiseasesModel Eff-4_fold-1.pt',
    '../input/cassava-trained-models/with_torch_crossentropy_LeafDiseasesModel Eff-4_fold-5.pt',
    '../input/cassava-trained-models/LeafDiseasesModel Eff-4_fold-1.pt',
    '../input/cassava-trained-models/LeafDiseasesModel Eff-4_fold-2.pt',
    '../input/cassava-trained-models/LeafDiseasesModel Eff-4_fold-3.pt',
    '../input/cassava-trained-models/LeafDiseasesModel Eff-4_fold-4.pt',
    '../input/cassava-trained-models/LeafDiseasesModel Eff-4_fold-5.pt',
    '../input/cassava-trained-models/LeafDiseasesModel Eff-7_fold-1.pt',
    '../input/cassava-trained-models/LeafDiseasesModel Eff-7_fold-2.pt',
    '../input/cassava-trained-models/LeafDiseasesModel Eff-7_fold-4.pt',
    '../input/cassava-trained-models/Resnext50_fold-1.pth',
    '../input/cassava-trained-models/Resnext50_fold-2.pth',
    '../input/cassava-trained-models/Resnext50_fold-3.pth',
    '../input/cassava-trained-models/Resnext50_fold-4.pth',
    '../input/cassava-trained-models/Resnext50_fold-5.pth',
    '../input/cassava-trained-models/Resnext50_with_lr_scheduler_fold-1.pth',
    '../input/cassava-trained-models/Resnext50_with_lr_scheduler_fold-2.pth',
    '../input/cassava-trained-models/Resnext50_with_lr_scheduler_fold-3.pth'
]

model = EnsembledModel(model_paths)

# Run Inference

In [ ]:
def inference(model, data_loader):

    epoch_preds = 0
    for epoch in range(CFG.epochs):

        preds = []
        for images in tqdm(data_loader):
            images = images.to(device)
            logits = model.predict(images)
            preds += [logits.softmax(1).detach().cpu().numpy()]
        all_img_preds = np.concatenate(preds, axis=0)
        epoch_preds += all_img_preds

    epoch_preds = epoch_preds / CFG.epochs
    return epoch_preds

In [ ]:
test = pd.read_csv('../input/cassava-leaf-disease-classification/sample_submission.csv')

#test_data = TestDataset(test, transform=test_aug)       # Use for TTA
test_data = TestDataset(test, transform=get_augmentation(data='test'))

test_loader = DataLoader(test_data, batch_size=CFG.batch_size, num_workers=CFG.num_workers)

In [ ]:
predictions = inference(model, test_loader)

# Submission

In [ ]:
test['label'] = predictions.argmax(1)
test[['image_id', 'label']].to_csv(OUTPUT_DIR + 'submission.csv', index=False)

# Conclusion

### Upvote the notebook if you like it.

### To go forward from here, click the blue "Edit Notebook" button at the top of the kernel. This will create a copy of the code and environment for you to edit. Delete, modify, and add code as you please. Happy Kaggling!